In [16]:
import pandas as pd
import numpy as np
import os
import datetime
import pinyin


# 设定路径
rpath = 'D:\\maran10\\Documents\\日常工作\\1.商机表\\9.25商机反馈\\'
wpath = 'D:\\maran10\\Documents\\日常工作\\1.商机表\\'
sep_path = wpath+'9.25商机拆分\\'

opp = pd.read_excel(wpath+'全国商用线下商机表9.18.xlsx',encoding = 'gbk')

# 下面这里是处理第一行的注释
# opp.columns = opp.iloc[0]  
# opp = opp.loc[1:,:]

# opp.head(10)  # 看一下原表格


In [17]:
# 循环读各区域提报的新商机表

count=0  # 计循环次数
files = os.listdir(rpath)
for f in files: 
    if not ".xlsx" in f:
        continue
    opp_region = pd.read_excel(rpath+f,encoding = "gbk")
    
#     opp_region.columns = opp_region.iloc[0]
#     opp_region = opp_region.loc[1:,:]

    count = count+1
#     print(opp_region.head(5))
    print("第",count,"次循环")
    print(f,"该文件中的数据条数",len(opp_region))
    
    # 直接筛选更新了的，没更新的不用管

    opp_new = opp_region[opp_region['本周是否更新']=="是"]
    print(f,"更新数据条数",len(opp_new))
    
    if count == 1:  # 第一次要先生成空表才能concat
        opp_renew = pd.DataFrame(columns = list(opp_new))  # 项目唯一编号不为空的，是更新商机
        opp_totalnew = pd.DataFrame(columns = list(opp_new))  # 项目唯一编号为空的，是新增商机

    opp_renew = pd.concat([opp_renew,opp_new[pd.notnull(opp_new['项目唯一编号\n（总部填写）'])]].copy())
    opp_totalnew = pd.concat([opp_totalnew,opp_new[pd.isnull(opp_new['项目唯一编号\n（总部填写）'])]].copy())
    
    print("更新的项目条数",len(opp_renew))
    print("新增的项目条数",len(opp_totalnew))
    
#### 写出表格
# opp_renew.to_excel(wpath+'全国更新的商机项目.xlsx',index=False,encoding='gbk')
# opp_totalnew.to_excel(wpath+'全国新增的商机项目.xlsx',index=False,encoding='gbk')

第 1 次循环
华东区-商用商机汇总表-20200924-更新.xlsx 该文件中的数据条数 9999
华东区-商用商机汇总表-20200924-更新.xlsx 更新数据条数 12
更新的项目条数 1
新增的项目条数 11
第 2 次循环
华中商用线下商机表9.24.xlsx 该文件中的数据条数 9999
华中商用线下商机表9.24.xlsx 更新数据条数 5
更新的项目条数 1
新增的项目条数 16
第 3 次循环
华北商用线下商机表9.24.xlsx 该文件中的数据条数 9999
华北商用线下商机表9.24.xlsx 更新数据条数 80
更新的项目条数 30
新增的项目条数 67
第 4 次循环
华南商用线下商机表-华南区汇总0924.xlsx 该文件中的数据条数 10000
华南商用线下商机表-华南区汇总0924.xlsx 更新数据条数 12
更新的项目条数 30
新增的项目条数 79
第 5 次循环
商用东北区商机--9月25号反馈.xlsx 该文件中的数据条数 9998
商用东北区商机--9月25号反馈.xlsx 更新数据条数 5
更新的项目条数 31
新增的项目条数 83


In [18]:
# 给新增的项目编号
def getStrAllAplha(str):
    return pinyin.get_initial(str, delimiter="").upper()
    
def getStrFirstAplha(str):
    str=getStrAllAplha(str)
    str=str[0:2]
    return str.upper()


opp_totalnew['today'] = [datetime.datetime.now().strftime('%Y%m%d')]*len(opp_totalnew)
opp_totalnew['大区首字母'] = opp_totalnew['*大区'].apply(lambda x:getStrFirstAplha(x))
opp_totalnew['业务分类首字母'] = opp_totalnew['*业务分类'].apply(lambda x:getStrFirstAplha(x))
opp_totalnew['序号'] = np.arange(1,len(opp_totalnew)+1)
opp_totalnew['序号'] = opp_totalnew['序号'].apply(lambda x:str(x))
opp_totalnew['项目唯一编号\n（总部填写）'] = opp_totalnew['today']+opp_totalnew['大区首字母']+opp_totalnew['业务分类首字母']+opp_totalnew['序号'].apply(lambda x:x.zfill(2))
opp_totalnew = opp_totalnew.drop(columns = ['today','大区首字母','业务分类首字母'])
# opp_totalnew.head(3)

In [19]:
a = opp['项目唯一编号\n（总部填写）'].value_counts()
print("项目唯一编号重复量",len(a[a>1])) # 这里应该为0
# 这里的去重只是为了保险检查
opp_dp = opp.drop_duplicates(subset=['项目唯一编号\n（总部填写）'],keep='first', inplace=False)
print(len(opp_dp))

项目唯一编号重复量 0
2129


In [20]:
# 将更新的项目表，和编码后的新增的项目表更新回总表
print("原全国商机表数据条数",len(opp_dp))

opp_new = pd.concat([opp_dp,opp_renew,opp_totalnew])
print("加上更新与新增商机后商机表数据条数",len(opp_new))

cou = opp_new['项目唯一编号\n（总部填写）'].value_counts()
# print(cou[cou>1])
print("重复条数",len(cou[cou>1]))  # 重复条数与更新条数应该相同

原全国商机表数据条数 2129
加上更新与新增商机后商机表数据条数 2243
重复条数 31


In [26]:
opp_new_dp = opp_new.drop_duplicates(subset=['项目唯一编号\n（总部填写）'],keep='last', inplace=False)
print("去重后全国商机表数据条数",len(opp_new_dp))

opp_new_dp = opp_new_dp[opp_new_dp['*项目状态']!='无效关闭']  # 删掉无效关闭的
print("去掉无效关闭后全国商机表数据条数",len(opp_new_dp))

opp_new_dp['*项目获取时间'] = opp_new_dp['*项目获取时间'].apply(lambda x:str(x))
opp_new_dp['*项目获取时间'] = opp_new_dp['*项目获取时间'].apply(lambda x: x[0:11])
opp_new_dp = opp_new_dp.sort_values('*项目获取时间',ascending=True)

# 重新生成序号
opp_new_dp['序号'] = np.arange(1,len(opp_new_dp)+1)

opp_new_dp['本周是否更新']=[np.nan]*len(opp_new_dp) # 这句话是去掉“是否更新”一列的“是”


# opp_new_dp['*项目获取时间'] = pd.to_datetime(opp_new['*项目获取时间'])
# opp_new_dp['*项目获取时间'] = opp_new_dp['*项目获取时间'].apply(lambda x: x.strftime('%Y%m%d'))  # 把日期列转换为日期格式

# opp_new_dp['*项目中标时间'] = pd.to_datetime(opp_new['*项目中标时间'])
# opp_new_dp['*项目中标时间'] = opp_new_dp['*项目中标时间'].apply(lambda x: x.strftime('%Y%m%d')) 

print("更新后全国商用线下商机表数据条数",len(opp_new_dp))

opp_new_dp.to_excel(wpath+'更新后全国商用线下商机表.xlsx',index=False,encoding='gbk')

去重后全国商机表数据条数 2212
去掉无效关闭后全国商机表数据条数 2212
更新后全国商用线下商机表数据条数 2212


In [27]:
# 按区域拆分
# opp_new = pd.read_excel(wpath+'全国商用线下商机表9.11.xlsx',encoding='gbk')
for name, group in opp_new_dp.groupby('*大区'):
    print(name)
    group.to_excel(sep_path+name+"商用线下商机表9.25.xlsx",encoding='gbk',index=False)

东北
华东
华中
华北
华南
西南
